딥러닝 학습 - 손실 함수를 최소화하는 인공신경망의 가중치와 편향을 찾는 과정

뇌 신경은 수많은 신경세포(뉴런)들이 연결되어 정보를 처리하고 전달한다

신경세포는 신호를 입력받아 다음 신경세포에 출력 형태로 연결한다는 점에서 입력과 출력을 갖는 함수와 비슷하다

인공 신경망은 이처럼 수많은 신경세포가 연결되는 뇌 신경계와 같이 수많은 함수를 서로 연결하여 복잡한 정보를 처리하는 네트워크 구조로 이해할 수 있다.

인공 신경망은 입력값을 가중치와 곱하고 그 결과를 전부 더함.

가중 합산한 값을 활성화 함수에 입력하여 최종 출력으로 변환

입력값들의 수학적 선형 결합을 다양한 형태의 비선형(또는 선형) 결합으로 변환하는 역할


어떤 활성화 함수를 적용하는지에 따라 딥러닝의 예측력에 상당한 차이가 발생한다

문제 유형 - 목표 레이블 - 활성화 함수

회귀 예측 - 연속형(숫자형) - 선형(linear)

이진 분류 - 이산형(범주형) - 시그모이드(sigmoid)

다중 분류 - 이산형(범주형) - 소프트맥스(softmax)

손실함수 J(W)

-인공 신경망이 출력하는 값과 실제 정답과의 차이를 말함

-손실 함수가 작을수록 좋다

-출력값과 실제값을 비교하여 차이를 최소화하는 가중치(w)와 편향의 조합을 찾자

<손실함수>

회귀예측: 평균제곱오차(mean_quared_error)

이진분류: 이진 교차 엔트로피(binary_crossentropy)

다중분류: 범주형 교차 엔트로피(categorical_crossentropy)

경사하강법

(손실함수가 아래로 볼록한 형태의 2차 함수의 경우)
미분해서 0인 지점이 손실이 가장 작은 가중치(W*)이다.

(손실함수가 복잡할 경우)
미분이 0이 되는 값이 여러 개 존재하게 되므로 미분만으로 최소값을 찾기 어렵기에 경사하강법!

경사하강법은 손실 함수의 현재 위치에서 조금씩 손실이 낮아지는 쪽으로 가중치를 이동하면서 최소값을 찾는 방법

Optimizer (손실함수를 최소화하는 방향으로 가중치를 갱신하는 알고리즘)

경사하강법에 기반을 둔 옵티마이저:

SGD, RMSProp, Adagrad, Adam

다층 신경망(MLP: Multi-Layer Perceptron)

인공 신경망은 일반적으로 여러 층의 레이어를 겹겹이 쌓아서 만듬

입력 레이어 - 은닉 레이어 - 출력 레이어

각 레이어는 여러 개의 유닛으로 구성

각 유닛은 하나의 퍼셉트론으로 이루어짐 또한 입력값과 가중치를 갖고 활성화 함수를 거쳐 출력을 보낸다.

인접한 두 레이어의 각 유닛은 서로 가중치를 갖는 연결선을 통해 이어지고 일대일 대응 관계를 갖는다.

In [4]:
#간단한 구조의 인공 신경망으로 1차 선형회귀식을 찾는 문제

import tensorflow as tf
print(tf.__version__)

2.15.0


입력할 데이터 준비

In [5]:
import pandas as pd
import numpy as np

#y=x+1 관계
x = [-3, 31,-11,4,0,22,-2,-5,-25,-14]
y = [-2,32,-10,5,1,23,-1,-4,-24,-13]

X_train = np.array(x).reshape(-1,1) #10행 1열
y_train = np.array(y) #1행

print(X_train.shape, y_train.shape)

(10, 1) (10,)


In [6]:
#1개의 레이어만 사용하여 가장 단순한 형태의 신경망을 만들어보자

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()  #모델 인스턴스 생성
model.add(Dense(units=1, activation = 'linear',input_dim = 1))

#add 메소드를 사용해 완전 연결 레이어(Dense)를 모델에 추가함. 완전 연결 레이어의 출력값은 목표 레이블(Y)을 예측한다.
#input_dim: 모델 학습에 사용하는 설명 변수(피처)의 개수를 지정한다. x가 하나이므로 1로 지정,
#units: 한 개의 연속형 수치를 예측하는 회귀 문제이므로 유닛 개수는 1이다.
#activation: 활성화 함수로 linear옵션을 지정하여 선형 함수의 출력을 그대로 사용

In [7]:
model.summary()

#param: 일차함수의 기울기(회귀계수), 절편(상수항) 총 2개

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2 (8.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
#모델 컴파일
#모델이 훈련하는데 필요한 기본 설정을 compile 함수에 지정한다.
#옵티마이저(adam 선택) 와 손실 함수(회귀니깐: 평균제곱오차 mse)를 설정
#metrics옵션 (보조 평가 지표): 평균절대오차(mae)를 추가하여 손실 함수를 모니터링할 때 함께 추적하기로 하자.

model.compile(optimizer='adam', loss= 'mse', metrics=['mae'])

#x 입력 (input_dim =1) -> 모델(param:w&b) 과 함께 linear -> output: 예측 이랑 정답이랑 비교 -> 손실함수 -> 옵티마이저 (가중치 갱신): 다시 모델로 빠꾸

모델 학습 및 예측

In [9]:
model.fit(X_train, y_train, epochs=3000, verbose=0)

#컴파일 단계에서 설정한 adam 옵티마이저와 mse 손실함수를 가지고 최적의 가중치와 편향을 찾는다.
#epochs: 전체 입력 데이터를 모두 몇 번 학습할 것인지 반복 횟수
#verbose: False(0)로 지정하면 훈련 과정을 화면에 보여주지 않는다.
#표시하려면 1 또는 2 입력

In [10]:
#학습을 마친 딥러닝 모델의 가중치를 확인
model.weights

#기울기에 해당하는 가중치(kernel:0) 값: 1
#절편에 해당하는 편향(bias: 0 ) 값: 1

#모델 학습을 통해 일차함수 관계식을 매우 근사하게 찾아낸 것으로 볼 수 있다.

[<tf.Variable 'dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.99999654]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.9993934], dtype=float32)>]

In [11]:
model.predict([[11],[12],[13]])

1/1 [==============================] - 0s 102ms/step


array([[11.999355 ],
       [12.9993515],
       [13.999349 ]], dtype=float32)